## SQL Window Functions Tutorial

### Introduction to Window Functions
- **What are Window Functions?**  
  Window functions perform calculations across a set of rows (a "window") related to the current row, without grouping the entire result set (unlike aggregate functions like `SUM` or `AVG`).
- **Key Features**:
  - Operate on a "window" of rows defined by `OVER` clause.
  - Preserve the original rows (unlike `GROUP BY`, which collapses rows).
  - Used for rankings, running totals, moving averages, etc.
- **Syntax**:
  ```sql
  function_name() OVER (
      [PARTITION BY column(s)]
      [ORDER BY column(s)]
      [ROWS or RANGE frame_spec]
  )
  ```
  - `PARTITION BY`: Divides data into partitions (like groups).
  - `ORDER BY`: Defines the order of rows within the window.
  - `ROWS`/`RANGE`: Specifies the subset of rows in the window.

### Common Window Functions
#### 1. Ranking Functions
- **ROW_NUMBER()**: Assigns a unique number to each row.
  ```sql
  SELECT name, grade,
         ROW_NUMBER() OVER (ORDER BY grade DESC) AS rank
  FROM students;
  ```
  - **Output Example** (assuming `students` table):
    ```
    name  | grade | rank
    Alice | 95.0  | 1
    Bob   | 85.5  | 2
    ```

- **RANK()**: Assigns a rank, with ties getting the same rank (gaps in numbering).
  ```sql
  SELECT name, grade,
         RANK() OVER (ORDER BY grade DESC) AS rank
  FROM students;
  ```
  - **Output Example** (if two students have grade 95.0):
    ```
    name  | grade | rank
    Alice | 95.0  | 1
    Bob   | 95.0  | 1
    Carol | 85.5  | 3
    ```

- **DENSE_RANK()**: Like `RANK()`, but no gaps in numbering for ties.
  ```sql
  SELECT name, grade,
         DENSE_RANK() OVER (ORDER BY grade DESC) AS rank
  FROM students;
  ```
  - **Output Example**:
    ```
    name  | grade | rank
    Alice | 95.0  | 1
    Bob   | 95.0  | 1
    Carol | 85.5  | 2
    ```

- **NTILE(n)**: Divides rows into `n` buckets.
  ```sql
  SELECT name, grade,
         NTILE(4) OVER (ORDER BY grade DESC) AS quartile
  FROM students;
  ```
  - Divides students into 4 groups based on grade.

#### 2. Aggregate Window Functions
- **SUM(), AVG(), COUNT(), MAX(), MIN()**: Compute aggregates over a window.
  - **Example**: Calculate running total of grades.
    ```sql
    SELECT name, grade,
           SUM(grade) OVER (ORDER BY id) AS running_total
    FROM students;
    ```
    - **Output Example**:
      ```
      name  | grade | running_total
      Alice | 95.0  | 95.0
      Bob   | 85.5  | 180.5
      ```

  - **Example**: Average grade per partition (e.g., by age group).
    ```sql
    SELECT name, age, grade,
           AVG(grade) OVER (PARTITION BY age) AS avg_grade_by_age
    FROM students;
    ```
    - Groups rows by `age` and computes the average grade within each group.

#### 3. Value Functions
- **LAG()**: Access the previous row’s value.
  ```sql
  SELECT name, grade,
         LAG(grade) OVER (ORDER BY id) AS previous_grade
  FROM students;
  ```
  - **Output Example**:
    ```
    name  | grade | previous_grade
    Alice | 95.0  | NULL
    Bob   | 85.5  | 95.0
    ```

- **LEAD()**: Access the next row’s value.
  ```sql
  SELECT name, grade,
         LEAD(grade) OVER (ORDER BY id) AS next_grade
  FROM students;
  ```

- **FIRST_VALUE()**, **LAST_VALUE()**: Get the first/last value in the window.
  ```sql
  SELECT name, grade,
         FIRST_VALUE(grade) OVER (PARTITION BY age ORDER BY grade DESC) AS highest_grade
  FROM students;
  ```

#### 4. Window Frame Specification
- **ROWS vs. RANGE**:
  - `ROWS`: Defines window based on a fixed number of rows.
  - `RANGE`: Defines window based on a range of values.
  - **Example**: Moving average of last 2 rows.
    ```sql
    SELECT name, grade,
           AVG(grade) OVER (
               ORDER BY id
               ROWS BETWEEN 1 PRECEDING AND CURRENT ROW
           ) AS moving_avg
    FROM students;
    ```
    - Computes average of the current row and the previous row.

### Practical Example in SQLite with Python


In [3]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('example.db')
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS students (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        age INTEGER,
        grade REAL
    )
''')

cursor.execute('DELETE FROM students')

sample_data = [
    ('Alice', 20, 95.0),
    ('Bob', 20, 85.5),
    ('Carol', 22, 90.0),
    ('Dave', 22, 88.0),
    ('Eve', 20, 92.0),
    ('Frank', 22, 87.0)
]
cursor.executemany('INSERT INTO students (name, age, grade) VALUES (?, ?, ?)', sample_data)
conn.commit()

# Function to execute query and return Pandas DataFrame
def execute_query_pandas(conn, query, title):
    try:
        df = pd.read_sql_query(query, conn)
        print(f"\n{title}:")
        # print(df)
        return df
    except sqlite3.Error as e:
        print(f"Query error: {e}")
        return pd.DataFrame()

In [5]:
# 1. ROW_NUMBER(): Unique row number
query1 = '''
    SELECT name, age, grade,
            ROW_NUMBER() OVER (ORDER BY grade DESC) AS row_num
    FROM students
'''
execute_query_pandas(conn, query1, "ROW_NUMBER (by grade descending)")


ROW_NUMBER (by grade descending):


,name,age,grade,row_num
0,Alice,20,95.0,1
1,Eve,20,92.0,2
2,Carol,22,90.0,3
3,Dave,22,88.0,4
4,Frank,22,87.0,5
5,Bob,20,85.5,6


In [6]:
# 2. RANK(): Rank with gaps for ties
query2 = '''
    SELECT name, age, grade,
            RANK() OVER (ORDER BY grade DESC) AS rank
    FROM students
'''
execute_query_pandas(conn, query2, "RANK (by grade descending)")


RANK (by grade descending):


,name,age,grade,rank
0,Alice,20,95.0,1
1,Eve,20,92.0,2
2,Carol,22,90.0,3
3,Dave,22,88.0,4
4,Frank,22,87.0,5
5,Bob,20,85.5,6


In [7]:
# 3. DENSE_RANK(): Rank without gaps for ties
query3 = '''
    SELECT name, age, grade,
            DENSE_RANK() OVER (ORDER BY grade DESC) AS dense_rank
    FROM students
'''
execute_query_pandas(conn, query3, "DENSE_RANK (by grade descending)")


DENSE_RANK (by grade descending):


,name,age,grade,dense_rank
0,Alice,20,95.0,1
1,Eve,20,92.0,2
2,Carol,22,90.0,3
3,Dave,22,88.0,4
4,Frank,22,87.0,5
5,Bob,20,85.5,6


In [ ]:
# 4. NTILE(4): Divide into 4 buckets
query4 = '''
    SELECT name, age, grade,
            NTILE(3) OVER (ORDER BY grade DESC) AS quartile
    FROM students
'''
execute_query_pandas(conn, query4, "NTILE(4) (by grade descending)")


NTILE(4) (by grade descending):


,name,age,grade,quartile
0,Alice,20,95.0,1
1,Eve,20,92.0,1
2,Carol,22,90.0,1
3,Dave,22,88.0,2
4,Frank,22,87.0,2
5,Bob,20,85.5,2


In [12]:
# 5. SUM(): Running total of grades
query5 = '''
    SELECT name, age, grade,
            SUM(grade) OVER (ORDER BY id) AS running_total
    FROM students
'''
execute_query_pandas(conn, query5, "SUM (Running total of grades)")


SUM (Running total of grades):


,name,age,grade,running_total
0,Alice,20,95.0,95.0
1,Bob,20,85.5,180.5
2,Carol,22,90.0,270.5
3,Dave,22,88.0,358.5
4,Eve,20,92.0,450.5
5,Frank,22,87.0,537.5


In [13]:
# 6. AVG(): Average grade per age group
query6 = '''
    SELECT name, age, grade,
            AVG(grade) OVER (PARTITION BY age) AS avg_grade_by_age
    FROM students
'''
execute_query_pandas(conn, query6, "AVG (by age group)")


AVG (by age group):


,name,age,grade,avg_grade_by_age
0,Alice,20,95.0,90.833333
1,Bob,20,85.5,90.833333
2,Eve,20,92.0,90.833333
3,Carol,22,90.0,88.333333
4,Dave,22,88.0,88.333333
5,Frank,22,87.0,88.333333


In [14]:
# 7. COUNT(): Count of students per age group
query7 = '''
    SELECT name, age, grade,
            COUNT(*) OVER (PARTITION BY age) AS count_in_age_group
    FROM students
'''
execute_query_pandas(conn, query7, "COUNT (by age group)")


COUNT (by age group):


,name,age,grade,count_in_age_group
0,Alice,20,95.0,3
1,Bob,20,85.5,3
2,Eve,20,92.0,3
3,Carol,22,90.0,3
4,Dave,22,88.0,3
5,Frank,22,87.0,3


In [15]:
# 8. MAX(): Maximum grade per age group
query8 = '''
    SELECT name, age, grade,
            MAX(grade) OVER (PARTITION BY age) AS max_grade_in_age
    FROM students
'''
execute_query_pandas(conn, query8, "MAX (by age group)")


MAX (by age group):


,name,age,grade,max_grade_in_age
0,Alice,20,95.0,95.0
1,Bob,20,85.5,95.0
2,Eve,20,92.0,95.0
3,Carol,22,90.0,90.0
4,Dave,22,88.0,90.0
5,Frank,22,87.0,90.0


In [16]:
# 9. MIN(): Minimum grade per age group
query9 = '''
    SELECT name, age, grade,
            MIN(grade) OVER (PARTITION BY age) AS min_grade_in_age
    FROM students
'''
execute_query_pandas(conn, query9, "MIN (by age group)")


MIN (by age group):


,name,age,grade,min_grade_in_age
0,Alice,20,95.0,85.5
1,Bob,20,85.5,85.5
2,Eve,20,92.0,85.5
3,Carol,22,90.0,87.0
4,Dave,22,88.0,87.0
5,Frank,22,87.0,87.0


In [17]:
# 10. LAG(): Previous grade in order
query10 = '''
    SELECT name, age, grade,
            LAG(grade) OVER (ORDER BY id) AS previous_grade
    FROM students
'''
execute_query_pandas(conn, query10, "LAG (previous grade by id)")


LAG (previous grade by id):


,name,age,grade,previous_grade
0,Alice,20,95.0,NaN
1,Bob,20,85.5,95.0
2,Carol,22,90.0,85.5
3,Dave,22,88.0,90.0
4,Eve,20,92.0,88.0
5,Frank,22,87.0,92.0


In [18]:
# 11. LEAD(): Next grade in order
query11 = '''
    SELECT name, age, grade,
            LEAD(grade) OVER (ORDER BY id) AS next_grade
    FROM students
'''
execute_query_pandas(conn, query11, "LEAD (next grade by id)")


LEAD (next grade by id):


,name,age,grade,next_grade
0,Alice,20,95.0,85.5
1,Bob,20,85.5,90.0
2,Carol,22,90.0,88.0
3,Dave,22,88.0,92.0
4,Eve,20,92.0,87.0
5,Frank,22,87.0,NaN


In [19]:
# 12. FIRST_VALUE(): First grade in each age group
query12 = '''
    SELECT name, age, grade,
            FIRST_VALUE(grade) OVER (PARTITION BY age ORDER BY grade DESC) AS highest_grade
    FROM students
'''
execute_query_pandas(conn, query12, "FIRST_VALUE (highest grade by age)")


FIRST_VALUE (highest grade by age):


,name,age,grade,highest_grade
0,Alice,20,95.0,95.0
1,Eve,20,92.0,95.0
2,Bob,20,85.5,95.0
3,Carol,22,90.0,90.0
4,Dave,22,88.0,90.0
5,Frank,22,87.0,90.0


In [23]:
# 13. LAST_VALUE(): Last grade in each age group
query13 = '''
    SELECT name, age, grade,
            LAST_VALUE(grade) OVER (
                PARTITION BY age
                ORDER BY grade
                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
            ) AS lowest_grade
    FROM students
'''
execute_query_pandas(conn, query13, "LAST_VALUE (lowest grade by age)")


LAST_VALUE (lowest grade by age):


,name,age,grade,lowest_grade
0,Bob,20,85.5,95.0
1,Eve,20,92.0,95.0
2,Alice,20,95.0,95.0
3,Frank,22,87.0,90.0
4,Dave,22,88.0,90.0
5,Carol,22,90.0,90.0


In [24]:
# 14. Frame Specification: Moving average of last 2 rows
query14 = '''
    SELECT name, age, grade,
            AVG(grade) OVER (
                ORDER BY id
                ROWS BETWEEN 1 PRECEDING AND CURRENT ROW
            ) AS moving_avg
    FROM students
'''
execute_query_pandas(conn, query14, "Moving Average (last 2 rows)")


Moving Average (last 2 rows):


,name,age,grade,moving_avg
0,Alice,20,95.0,95.00
1,Bob,20,85.5,90.25
2,Carol,22,90.0,87.75
3,Dave,22,88.0,89.00
4,Eve,20,92.0,90.00
5,Frank,22,87.0,89.50


In [25]:
# Close connection
cursor.close()
conn.close()